# LLMs are pure functions

In [2]:
from pydantic import BaseModel, Field
import json

from basic.api_key import ensure_api_key
ensure_api_key()
from openai import OpenAI
client = OpenAI()

OpenAI API key has been configured.


In [2]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="What is the current price of Bitcoin in USD?",
)
print(response.output_text)

I can't provide real-time data, including the current price of Bitcoin. To find the latest price, please check a financial news website, a cryptocurrency exchange, or a financial app.


# Calling functions

## Mock function

In [11]:
from basic.utils import get_current_price
get_current_price("EUR/USD")

Fetching current price for EUR/USD...


{'pair': 'EUR/USD',
 'base': 'EUR',
 'quote': 'USD',
 'price': 3669.5,
 'timestamp': '2025-10-01T15:27:41.821091'}

In [4]:
class ToolCall(BaseModel):
    tool_name: str = Field(description="The name of the tool to be called. The available tools are: get_current_price.")
    tool_input: str = Field(description="The input to be passed to the tool. For the get_current_price tool, this should be the currency pair, e.g., BTC/USD.")

class Reply(BaseModel):
    answer: str | None = Field(description="The answer to the question in a concise manner.")
    tool_call: ToolCall | None = Field(description="If the answer requires calling a tool, this field will contain the tool call details.")

In [5]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that is good with finance."},
    {"role": "user", "content": "What is the current price of Bitcoin in USD?"},
]
response = client.chat.completions.parse(
    model="gpt-4o-mini",
    messages=messages,
    response_format=Reply
)
response_text = response.choices[0].message.content
reply = Reply.model_validate_json(response_text)
messages.append({"role": "assistant", "content": response_text})
reply

Reply(answer=None, tool_call=ToolCall(tool_name='get_current_price', tool_input='BTC/USD'))

In [6]:
if reply.tool_call is not None:
    if reply.tool_call.tool_name == "get_current_price":
        current_price = get_current_price(reply.tool_call.tool_input)
        messages.append({"role": "user", "content": json.dumps(current_price)})
        response = client.chat.completions.parse(
            model="gpt-4o-mini",
            messages=messages,
            response_format=Reply
        )
        response_text = response.choices[0].message.content
        reply = Reply.model_validate_json(response_text)
        messages.append({"role": "assistant", "content": response_text})

        print(reply)
    else:
        print("Some other tool has been called.")
else:
    print("No luck this time. Try again.")

Fetching current price for BTC/USD...
answer='The current price of Bitcoin (BTC) is $53,368.70 USD.' tool_call=None


## OpenAI function calling

In [7]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_price",
            "description": "Get the current price of a cryptocurrency trading pair",
            "parameters": {
                "type": "object",
                "properties": {
                    "pair": {
                        "type": "string",
                        "description": "The trading pair in format BASE/QUOTE, e.g. BTC/USD, ETH/USD"
                    }
                },
                "required": ["pair"],
                "additionalProperties": False
            }
        }
    }
]

messages = [
    {"role": "system", "content": "You are a helpful assistant that is good with finance."},
    {"role": "user", "content": "What is the current price of Bitcoin in USD?"},
]

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages,
    tools=tools
)
response_message = response.choices[0].message
messages.append(response_message)

if response_message.tool_calls:
    tool_call = response_message.tool_calls[0]
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    if function_name == "get_current_price":
        function_response = get_current_price(**function_args)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(function_response)
        })
        resp = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages,
            tools=tools
        )
        resp_message = resp.choices[0].message
        print(resp_message.content)
        messages.append({"role": "assistant", "content": resp_message.content})

Fetching current price for BTC/USD...
The current price of Bitcoin (BTC) in USD is $29,760.84.


# Bonus material - how the function calling depends on the keywords like tools and functions

In [3]:
class AssistanceRequest(BaseModel):
    assistance_name: str = Field(description="The name of the assistance needed. The available assistance requests are: get_current_price.")
    assistance_input: str = Field(description="The input to be passed to the assistance handler. For the get_current_price assistance, this should be the currency pair, e.g., BTC/USD.")

class Reply(BaseModel):
    answer: str | None = Field(description="The answer to the question in a concise manner.")
    assistance_request: AssistanceRequest | None = Field(description="If the answer requires assistance, this field will contain the assistance request details.")

messages = [
    {"role": "system", "content": "You are a helpful assistant that is good with finance."},
    {"role": "user", "content": "What is the current price of Bitcoin in USD?"},
]
response = client.chat.completions.parse(
    model="gpt-4o-mini",
    messages=messages,
    response_format=Reply
)
response_text = response.choices[0].message.content
reply = Reply.model_validate_json(response_text)
messages.append({"role": "assistant", "content": response_text})
reply

Reply(answer=None, assistance_request=AssistanceRequest(assistance_name='get_current_price', assistance_input='BTC/USD'))

## The variations of the model and Reply object structure

You have to download it and pull the model llama2 first:

## GPT-4o-mini but with "assistance" instead of "tools" or "functions"

In [8]:
class AssistanceRequest(BaseModel):
    assistance_name: str = Field(description="The name of the assistance needed. The available assistance requests are: get_current_price.")
    assistance_input: str = Field(description="The input to be passed to the assistance handler. For the get_current_price assistance, this should be the currency pair, e.g., BTC/USD.")

class Reply(BaseModel):
    answer: str | None = Field(description="The answer to the question in a concise manner.")
    assistance_request: AssistanceRequest | None = Field(description="If the answer requires assistance, this field will contain the assistance request details.")

messages = [
    {"role": "system", "content": "You are a helpful assistant that is good with finance."},
    {"role": "user", "content": "What is the current price of Bitcoin in USD?"},
]
client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama'
)
response = client.chat.completions.parse(
    model="llama3.2",
    messages=messages,
    response_format=Reply
)
response_text = response.choices[0].message.content
reply = Reply.model_validate_json(response_text)
messages.append({"role": "assistant", "content": response_text})
reply

Reply(answer=None, assistance_request=AssistanceRequest(assistance_name='current_value_BTC', assistance_input="As of my knowledge cutoff in Dec 2023, however please note that cryptocurrency prices can fluctuate rapidly. I'm not aware of the current price. However , I can direct you to a reliable source to find out the up-to-date Bitcoin price in USD."))

## The llama3.2 model with "tools"

In [15]:
class ToolCall(BaseModel):
    tool_name: str = Field(description="The name of the tool to be called. The available tools are: get_current_price.")
    tool_input: str = Field(description="The input to be passed to the tool. For the get_current_price tool, this should be the currency pair, e.g., BTC/USD.")

class Reply(BaseModel):
    answer: str | None = Field(description="The answer to the question in a concise manner.")
    tool_call: ToolCall | None = Field(description="If the answer requires calling a tool, this field will contain the tool call details.")

messages = [
    {"role": "system", "content": "You are a helpful assistant that is good with finance."},
    {"role": "user", "content": "What is the current price of Bitcoin in USD?"},
]
client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama'
)
response = client.chat.completions.parse(
    model="llama3.2",
    messages=messages,
    response_format=Reply
)
response_text = response.choices[0].message.content
reply = Reply.model_validate_json(response_text)
messages.append({"role": "assistant", "content": response_text})
reply

Reply(answer="I'm not capable of sharing real time information but I can offer some resources for you to get the most up-to-date current information, such as looking at Crypto exchanges like Coinbase.", tool_call=ToolCall(tool_name='Bitcoin Price', tool_input='Check current price here https://www.coingecko.com/en'))

## Llama3.2 with "assistance"

In [16]:
class AssistanceRequest(BaseModel):
    assistance_name: str = Field(description="The name of the assistance needed. The available assistance requests are: get_current_price.")
    assistance_input: str = Field(description="The input to be passed to the assistance handler. For the get_current_price assistance, this should be the currency pair, e.g., BTC/USD.")

class Reply(BaseModel):
    answer: str | None = Field(description="The answer to the question in a concise manner.")
    assistance_request: AssistanceRequest | None = Field(description="If the answer requires assistance, this field will contain the assistance request details.")


messages = [
    {"role": "system", "content": "You are a helpful assistant that is good with finance."},
    {"role": "user", "content": "What is the current price of Bitcoin in USD?"},
]
client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama'
)
response = client.chat.completions.parse(
    model="llama3.2",
    messages=messages,
    response_format=Reply
)
response_text = response.choices[0].message.content
reply = Reply.model_validate_json(response_text)
messages.append({"role": "assistant", "content": response_text})
reply

Reply(answer="Unfortunately, I'm not able to provide real-time data as my knowledge cutoff is in December 2023. However, I can suggest some ways for you to find the current price of Bitcoin in USD. You can check websites such as CoinMarketCap or CryptoCompare for the latest prices.", assistance_request=AssistanceRequest(assistance_name='Bitcoin Price', assistance_input='Please pass no input'))

## Llama3.2 with official tools parameter

In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_price",
            "description": "Get the current price of a cryptocurrency trading pair",
            "parameters": {
                "type": "object",
                "properties": {
                    "pair": {
                        "type": "string",
                        "description": "The trading pair in format BASE/QUOTE, e.g. BTC/USD, ETH/USD"
                    }
                },
                "required": ["pair"],
                "additionalProperties": False
            }
        }
    }
]

messages = [
    {"role": "system", "content": "You are a helpful assistant that is good with finance."},
    {"role": "user", "content": "What is the current price of Bitcoin in USD?"},
]
client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama'
)
response = client.chat.completions.create(
    model="llama3.2",
    messages=messages,
    tools=tools
)
response_message = response.choices[0].message
messages.append(response_message)

if response_message.tool_calls:
    tool_call = response_message.tool_calls[0]
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    if function_name == "get_current_price":
        function_response = get_current_price(**function_args)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(function_response)
        })
        resp = client.chat.completions.create(
            model="llama3.2",
            messages=messages,
            tools=tools
        )
        resp_message = resp.choices[0].message
        print(resp_message.content)
        messages.append({"role": "assistant", "content": resp_message.content})

Fetching current price for BTC/USD...
The current price of Bitcoin in USD is $46,859.12. Please note that this price may have changed since the timestamp provided and I'm just providing an approximate value based on the output of my tool. You can check a more up-to-date price using other sources such as coin market caps or financial websites.
